In [1]:
from data_functions import *

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import statsmodels.formula.api as smf

import random

In [2]:
for_pres = read('texas_president.xlsx')
for_pres = for_pres.rename(index=str, columns={"count": "county"})
for_pres = for_pres.rename(index=str, columns={"Urbanization Status": "urban", "Border Status": "border"})

In [3]:
dist_count = read('main_district_for_counties.csv')

In [4]:
temp_pres = for_pres.merge(dist_count, left_on='county', right_on ='Unnamed: 0')

In [5]:
counties = list(for_pres.county)

county_nums = {}
val = 1
for c in counties:
    county_nums[c] = val
    val += 1

'''
for_house['county_num'] = for_house.county.apply(lambda x: county_nums[x])
for_house['office'] = 'house'
for_house = for_house[(for_house.two_party_vtshare != 1) & (for_house.two_party_vtshare != 0)]
'''

"\nfor_house['county_num'] = for_house.county.apply(lambda x: county_nums[x])\nfor_house['office'] = 'house'\nfor_house = for_house[(for_house.two_party_vtshare != 1) & (for_house.two_party_vtshare != 0)]\n"

## COUNTY INFO

In [6]:
county_info = for_pres[['county', 'county_num', 'urban', 'border']]
county_info['urban'] = county_info.urban.apply(lambda x: 1 if x == 'Urban' else 0)
county_info['border'] = county_info.urban.apply(lambda x: 1 if x == 'Border' else 0)

In [7]:
county_info.border.value_counts()

0    254
Name: border, dtype: int64

In [8]:
cd = read('demo.csv')
cd = cd.drop(['Unnamed: 0','FIPS','Age'], axis=1)[1:]
cd.County = cd.County.apply(lambda x: x.split(" ")[0])
cd = cd[['County', 'Total', 'Total Female', 'Hispanic Total']]
cd['percent_female'] = cd['Total Female'] / cd['Total']
cd['percent_hispanic'] = cd['Hispanic Total'] / cd['Total']

cd.columns = ['county', 'population', 'Total Female', 'Hispanic Total', 'percent_female','percent_hispanic']
cd = cd[['county', 'population', 'percent_female','percent_hispanic']]
county_info = county_info.merge(cd)

vc = read('vets_and_crimes.xlsx')
vc.county = vc.county.apply(lambda x: x.upper())
county_info = county_info.merge(vc)
county_info = county_info.drop(['crimes_per_100k'], axis=1)

## NEXT INFO

In [9]:
temp_pres = temp_pres.rename(index=str, columns={"1": "district_num"})

In [10]:
temp_pres.head().style

,county,county_num,office,dem_candidate,two_party_vtshare,candidate_male,candidate_white,candidate_hispanic,urban,border,pro_gun,pro_choice,climate_pro,Unnamed: 0,district_num
0,ANDERSON,1,President,Hillary Clinton,0.203319,0,1,0,Rural,Non-Border,0,1,1,ANDERSON,5
1,ANDREWS,2,President,Hillary Clinton,0.17552,0,1,0,Rural,Non-Border,0,1,1,ANDREWS,11
2,ANGELINA,3,President,Hillary Clinton,0.258098,0,1,0,Rural,Non-Border,0,1,1,ANGELINA,1
3,ARANSAS,4,President,Hillary Clinton,0.241548,0,1,0,Urban,Non-Border,0,1,1,ARANSAS,27
4,ARCHER,5,President,Hillary Clinton,0.0942584,0,1,0,Urban,Non-Border,0,1,1,ARCHER,13


In [11]:
temp_pres = temp_pres[['county', 'office', 'district_num', 'dem_candidate', 'two_party_vtshare', 
                       'county_num', 'office', 'pro_gun',
                       'candidate_male', 'candidate_hispanic']]

In [12]:
temp_pres.head()

,county,office,district_num,dem_candidate,two_party_vtshare,county_num,office,pro_gun,candidate_male,candidate_hispanic
0,ANDERSON,President,5,Hillary Clinton,0.203319,1,President,0,0,0
1,ANDREWS,President,11,Hillary Clinton,0.175520,2,President,0,0,0
2,ANGELINA,President,1,Hillary Clinton,0.258098,3,President,0,0,0
3,ARANSAS,President,27,Hillary Clinton,0.241548,4,President,0,0,0
4,ARCHER,President,13,Hillary Clinton,0.094258,5,President,0,0,0


In [13]:
temp_pres.to_csv('add_pres.csv')

## TO ADD SENATE

In [14]:
# Should only have to add district via this line
senate = for_pres.merge(dist_count, left_on='county', right_on ='Unnamed: 0')
# and then candidate_male and candidate_hispanic